In [95]:
from copy import deepcopy
from aocd.models import Puzzle
puzzle = Puzzle(year=2021, day=23)
data = puzzle.input_data.split('\n')

In [93]:
data = '''
#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########
'''
data = data.split('\n')[1:-1]

In [34]:
def free_neighbours(hallway, i):
    for j in range(i + 1, len(hallway)):
        if hallway[j] == 0:
            yield j
        else:
            break
    for j in range(i - 1, - 1, -1):
        if hallway[j] == 0:
            yield j
        else:
            break

class Game:
    def __init__(self, hallway, rooms):
        self.hallway = hallway
        self.rooms = rooms

    @classmethod
    def from_string(cls, s):
        hallway = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        rooms = [['0ABCD'.index(s[i][j]) for i in [2, 3]] for j in [3, 5, 7, 9]]
        return cls(hallway, rooms)

    def __repr__(self):
        lines = [
            ''.join(str(h) for h in self.hallway),
            '  {0} {1} {2} {3}  '.format(*[r[0] for r in self.rooms]),
            '  {0} {1} {2} {3}  '.format(*[r[1] for r in self.rooms]),
        ]
        return '\n'.join(lines)

    def __eq__(self, other):
        return self.hallway == other.hallway and self.rooms == other.rooms

    def __hash__(self):
        h = tuple(self.hallway)
        r = tuple([tuple(rr) for rr in self.rooms])
        return hash((h, r))

    def finished(self):
        return self.rooms == [[1, 1], [2, 2], [3, 3], [4, 4]]

    def valid_moves(self):
        for i, h in enumerate(self.hallway):
            if h > 0:
                room_free = self.rooms[h-1] == [0, 0] or self.rooms[h-1] == [0, h]
                path = [p for x, p in enumerate(self.hallway) if i < x <= 2 * h or 2 * h <= x < i]
                path_free = not any(path)
                if room_free and path_free:
                    cost = len(path) * 10 ** (h - 1)
                    new = deepcopy(self)
                    new.hallway[i] = 0
                    if self.rooms[h-1][1] == 0:
                        cost += 2 * 10 ** (h - 1)
                        new.rooms[h-1][1] = h
                    else:
                        cost += 10 ** (h - 1)
                        new.rooms[h-1][0] = h
                    yield new, cost
        for i, r in enumerate(self.rooms):
            j = 2 * (i + 1)
            if self.hallway[j] > 0:
                continue
            if r == [i+1, i+1]:
                continue
            room_cost = 0
            if r[0] > 0:
                v = r[0]
                room_cost += 10 ** (v - 1)
                room_i = 0
            elif r[1] > 0 and r[1] != i + 1:
                v = r[1]
                room_cost += 2 * 10 ** (v - 1)
                room_i = 1
            if room_cost > 0:
                neighbours = free_neighbours(self.hallway, j)
                for k in free_neighbours(self.hallway, j):
                    if k / 2 in [1, 2, 3, 4]:
                        continue
                    cost = room_cost + abs(j - k) * 10 ** (v - 1)
                    new = deepcopy(self)
                    new.hallway[k] = v
                    new.rooms[i][room_i] = 0
                    yield new, cost


game = Game.from_string(data)
print(game)

costs = {game: 0}
class Node:
    def __init__(self, game, parent=None, cost=None, explored=False):
        self.game = game
        self.parent = parent
        self.children = []
        self.visited = False
        self.explored = explored
        self.cost = cost

    def visit(self):
        if self.visited:
            # visit one of the children
            for child in self.children:
                if not child.explored:
                    child.visit()
                    break
            else:
                self.explored = True
        else:
            self.visited = True
            if self.game.finished():
                self.explored = True
                return
            for new_game, cost in self.game.valid_moves():
                cumulative_cost = costs[self.game] + cost
                if new_game in costs:
                    if costs[new_game] > cumulative_cost:
                        costs[new_game] = cumulative_cost
                    self.children.append(Node(new_game, self, cost=cost, explored=True))
                else:
                    costs[new_game] = cumulative_cost          
                    self.children.append(Node(new_game, self, cost=cost))
            if len(self.children) == 0:
                self.explored = True

root = Node(game)
while not root.explored:
    root.visit()

def update(node):
    if node.parent and costs[node.game] > (costs[node.parent.game] + node.cost):
        costs[node.game] = costs[node.parent.game] + node.cost
    for child in node.children:
        update(child)

old_costs = None
while old_costs != costs:
    old_costs = deepcopy(costs)
    update(root)

win = Game([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [[1, 1], [2, 2], [3, 3], [4, 4]])
print(f'Least energy required: {costs[win]}.')

00000000000
  2 3 1 4  
  2 3 4 1  
Least energy required: 11120.


In [ ]:
puzzle.answer_a = 11120

In [96]:
def free_neighbours(hallway, i):
    for j in range(i + 1, len(hallway)):
        if hallway[j] == 0:
            yield j
        else:
            break
    for j in range(i - 1, - 1, -1):
        if hallway[j] == 0:
            yield j
        else:
            break

class Game:
    def __init__(self, hallway, rooms):
        self.hallway = hallway
        self.rooms = rooms

    @classmethod
    def from_string(cls, s):
        hallway = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        rooms = [['0ABCD'.index(s[i][j]) for i in [2, 3, 4, 5]] for j in [3, 5, 7, 9]]
        return cls(hallway, rooms)

    def __repr__(self):
        lines = [
            ''.join(str(h) for h in self.hallway),
            '  {0} {1} {2} {3}  '.format(*[r[0] for r in self.rooms]),
            '  {0} {1} {2} {3}  '.format(*[r[1] for r in self.rooms]),
            '  {0} {1} {2} {3}  '.format(*[r[2] for r in self.rooms]),
            '  {0} {1} {2} {3}  '.format(*[r[3] for r in self.rooms]),
        ]
        return '\n'.join(lines)

    def __hash__(self):
        h = tuple(self.hallway)
        r = tuple([tuple(rr) for rr in self.rooms])
        return hash((h, r))

    def __eq__(self, other):
        return hash(self) == hash(other)

    def finished(self):
        return self.rooms == [[i] * 4 for i in [1, 2, 3, 4]]

    def valid_moves(self):
        for i, h in enumerate(self.hallway):
            if h > 0:
                room_free = set(self.rooms[h-1]).issubset(set([0, h]))
                path = [p for x, p in enumerate(self.hallway) if i < x <= 2 * h or 2 * h <= x < i]
                path_free = not any(path)
                if room_free and path_free:
                    cost = len(path) * 10 ** (h - 1)
                    new = deepcopy(self)
                    new.hallway[i] = 0
                    num_present = self.rooms[h-1].count(h)
                    cost += (4 - num_present) * 10 ** (h - 1)
                    new.rooms[h-1][3-num_present] = h
                    yield new, cost
        for i, r in enumerate(self.rooms):
            j = 2 * (i + 1)
            if self.hallway[j] > 0:
                continue
            if set(r) == set([i + 1]):
                continue
            if set(r) == set([0]):
                continue
            room_i, v = next((i, v) for i, v in enumerate(r) if v > 0)
            room_cost = (room_i + 1) * 10 ** (v - 1)
            neighbours = free_neighbours(self.hallway, j)
            for k in free_neighbours(self.hallway, j):
                if k / 2 in [1, 2, 3, 4]:
                    continue
                cost = room_cost + abs(j - k) * 10 ** (v - 1)
                new = deepcopy(self)
                new.hallway[k] = v
                new.rooms[i][room_i] = 0
                yield new, cost

fold = """
  #D#C#B#A#
  #D#B#A#C#
"""
fold = fold.split('\n')[1:-1]
game = Game.from_string(data[:3] + fold + data[3:])
# game.rooms = [
#     [2, 2, 2, 2],
#     [1, 1, 1, 1],
#     [3, 3, 3, 3],
#     [4, 4, 4, 4],
# ]
print(game)

00000000000
  2 3 1 4  
  4 3 2 1  
  4 2 1 3  
  2 3 4 1  


In [97]:
costs = {game: 0}
class Node:
    def __init__(self, game, parent=None, cost=None, explored=False):
        self.game = game
        self.parent = parent
        self.children = []
        self.visited = False
        self.explored = explored
        self.cost = cost

    def visit(self):
        if self.visited:
            # visit one of the children
            for child in self.children:
                if not child.explored:
                    child.visit()
                    break
            else:
                self.explored = True
        else:
            self.visited = True
            if self.game.finished():
                self.explored = True
                return
            for new_game, cost in self.game.valid_moves():
                cumulative_cost = costs[self.game] + cost
                if new_game in costs:
                    if costs[new_game] > cumulative_cost:
                        costs[new_game] = cumulative_cost
                    self.children.append(Node(new_game, self, cost=cost, explored=True))
                else:
                    costs[new_game] = cumulative_cost          
                    self.children.append(Node(new_game, self, cost=cost))
            if len(self.children) == 0:
                self.explored = True

root = Node(game)
while not root.explored:
    root.visit()

def update(node):
    if node.parent and costs[node.game] > (costs[node.parent.game] + node.cost):
        costs[node.game] = costs[node.parent.game] + node.cost
    for child in node.children:
        update(child)

old_costs = None
while old_costs != costs:
    old_costs = deepcopy(costs)
    update(root)

win = Game([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [[i] * 4 for i in [1, 2, 3, 4]])
print(f'Least energy required: {costs[win]}.')

Least energy required: 49232.


In [ ]:
puzzle.answer_b = 49232